In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
import pandas as pd

In [2]:
N=100

In [3]:
mean_0 = np.zeros(5)
mean_1 = np.ones(5)
mean_2 = np.array((-2,2,-2,2,-2))
mean_3 = np.array((1/2,1,2,3,4))
cov_0 = np.eye(5)
cov_1 = np.eye(5)*1/2
cov_2 = np.eye(5)
cov_3 = np.eye(5)
for i in range(5):
    cov_2[i,i] = 1/(i+1)
    cov_3[i,i] = 0.5*(i+0.01)

mean_list = [mean_0,mean_1,mean_2,mean_3]
cov_list = [cov_0,cov_1,cov_2,cov_3]

In [4]:
dict_gaussian = {}
for i in range(4):
    dict_gaussian['gaussian_{0}'.format(i)] = np.random.multivariate_normal(mean=mean_list[i], cov=cov_list[i], size=N)

In [5]:
whole_data = np.empty((0,5))

In [6]:
for i in dict_gaussian:
    whole_data = np.vstack((whole_data,dict_gaussian[i]))

In [7]:
true_labels = np.ones((4*N,1))
observed_labels = np.ones((4*N,1))

In [8]:
for i in range(N):
    true_labels[i,0] = 0
    true_labels[N+i,0] = 1
    true_labels[2*N+i,0] = 2
    true_labels[3*N+i,0] = 3

In [9]:
whole_data = np.hstack((whole_data,true_labels,observed_labels))

In [10]:
for i in range(int((4*N)/2)):
    whole_data[2*i,6] = 99
    whole_data[2*i+1,6] = whole_data[2*i+1,5]

So now, half of the dataset is unlabeled.
We will thus try to predict the most probable category.

In [11]:
df_whole_data = pd.DataFrame(whole_data)

In [12]:
df_whole_data.columns = ['x_1','x_2','x_3','x_4','x_5','true_label','observed_label']

In [13]:
estimation_mean = df_whole_data.drop(['true_label'],axis=1).groupby('observed_label').mean().to_numpy()

In [14]:
estimation_cov = {}
for i in range(4):
    partial_data_cov = np.cov(df_whole_data[df_whole_data['observed_label'] == i].drop(['true_label','observed_label'],axis=1).to_numpy(),
                             rowvar=False)
    estimation_cov[str(i)] = partial_data_cov

In [15]:
dict_pi = {}
for i in range(4):
    dict_pi['pi_{0}'.format(i)] = 1/4

In [16]:
iterations = 1000

In [17]:
weights = np.ones((4*N,4))    

In [18]:
def update_cov_4(X,mean,weights,group):
    """a function to estimate the covariance with a new mean
    Parameter
    -------------------------------
    X : array-like
    the data with which we want to estimate the new covariance

    mean : array-like
    the new mean that doesn't correspond to the 'true mean'

    weights : arrar-like 
    the matrix of weights of the whole data

    group : int
    the group in which we want to update
    --------------------------------
    """
    sum_of_mat = np.zeros((X.shape[1],X.shape[1]))
    for i in range(X.shape[0]):
        temporal_cov = weights[i,group]*np.matmul((X[i,:]-mean).reshape((X.shape[1],1)),
                                                      (X[i,:]-mean).reshape((1,X.shape[1])))
        sum_of_mat += temporal_cov
    sum_of_weights = np.sum(weights[:,group])
    weighted_sigma = sum_of_mat/sum_of_weights
    return weighted_sigma

In [19]:
n=0
while n<=iterations:
    n+=1
    for component in range(4):
        #update of weights
        for i in range(whole_data.shape[0]):
            if whole_data[i,6] == 99:
                x_test = whole_data[i,0:5]
                numerator = dict_pi['pi_{0}'.format(component)]*multivariate_normal.pdf(mean=estimation_mean[component,:],
                                                                                        cov=estimation_cov[str(component)],
                                                                                        x=x_test)
                denom = dict_pi['pi_0']*multivariate_normal.pdf(mean=estimation_mean[0,:],cov=estimation_cov['0'],x=x_test)+dict_pi['pi_1']*multivariate_normal.pdf(mean=estimation_mean[1,:],cov=estimation_cov['1'],x=x_test)+dict_pi['pi_2']*multivariate_normal.pdf(mean=estimation_mean[2,:],cov=estimation_cov['2'],x=x_test)+dict_pi['pi_3']*multivariate_normal.pdf(mean=estimation_mean[3,:],cov=estimation_cov['3'],x=x_test)
                data_point_w = numerator/denom
                weights[i,component] = data_point_w
            elif whole_data[i,6] == component:
                weights[i,component] = 1
            else:
                weights[i,component] = 0
        #update of parameters
        dict_pi['pi_{0}'.format(component)] = np.sum(weights[:,component])
        estimation_mean[component,:] = np.sum(whole_data[:,0:5]*(weights[:,component].reshape(whole_data.shape[0],1)))/np.sum(weights[:,component])
        estimation_cov[str(component)] = update_cov_4(X=whole_data[:,0:5],group=component,
                                                      mean=estimation_mean[component,:],weights=weights)
        
                                              
        
        
        

In [20]:
predicted_label = np.zeros((4*N,1))
for i in range(4*N):
    predicted_label[i,0] = np.where(weights[i,:] == np.max(weights[i,:]))[0][0]

In [21]:
well_labeled = 0
wrongly_labeled = 0
for i in range(4*N):
    if predicted_label[i,0] == whole_data[i,5]:
        well_labeled += 1
    else:
        wrongly_labeled += 1

percentage_well_labeled = well_labeled/4
error_percentage = wrongly_labeled/4

In [22]:
print('good prediction :', percentage_well_labeled, '\n', 'error rate :', error_percentage)

good prediction : 82.75 
 error rate : 17.25


## For a hundred iterations, the well labeled points corresponded to 83.25% of the dataset, the error rate was thus 16.75%

With 1000 iterations it is worse : good prediction : 82.75 
 error rate : 17.25